# 1. 매수,매도 로직

In [1]:
'''

** 로직 순서 **

1. 데이터 불러오기
2. 

'''

'\n\n** 로직 순서 **\n\n1. 데이터 불러오기\n2. \n\n'

# 기술적 지표

In [2]:
# 기술적 지표 기본 클래스
class Technical_indicators:
    '''
    기술적 지표 클래스 (바이낸스 기준)
    
    '''
    def __init__(self, ticker, interval, cnt):
        self.ticker = ticker # 티커이름
        self.interval = interval # 인터벌
        self.cnt = cnt # 불러올 테이블 개수
        self.df = Technical_indicators.data_select(self)

    # 데이터 조회
    def data_select(self):
        ohlcv = exchange.fetch_ohlcv(symbol=self.ticker, timeframe=self.interval, limit= self.cnt)
        self.df = pd.DataFrame(ohlcv, columns=['datetime', 'open', 'high', 'low', 'close', 'volume'])
        pd_ts = pd.to_datetime(self.df['datetime'], utc=True, unit='ms')     # unix timestamp to pandas Timeestamp
        pd_ts = pd_ts.dt.tz_convert("Asia/Seoul")                       # convert timezone
        pd_ts = pd_ts.dt.tz_localize(None)
        self.df.set_index(pd_ts, inplace=True)
        self.df = self.df[['open', 'high', 'low', 'close', 'volume']]
        self.df.reset_index(inplace=True)
        date = self.df['datetime'].apply(lambda x:x.strftime('%Y-%m-%d'))
        hour = self.df['datetime'].apply(lambda x:x.strftime('%H')+':00:00')
        self.df['date'] = date
        self.df['hour'] = hour
        return self.df
    
    # RSI
    def RSI(self, timeperiod):
        self.df['rsi'] = talib.RSI(self.df['close'],timeperiod= timeperiod)
        self.df['rsi'] = self.df['rsi'].apply(lambda x:round(x,2))
#         self.df.dropna(inplace=True)
        # self.df.iloc[-1] = self.df.iloc[-2]
        return self.df

    # 스토캐스틱 RSI
    def STOCHRSI(self, timeperiod, d, m):
        self.df['close_stoch'] = talib.STOCHRSI(self.df["close"],timeperiod,fastk_period=timeperiod,fastd_period=d,fastd_matype=m)[-1]
        self.df['close_stoch'] = self.df['close_stoch'].apply(lambda x:round(x,2))
#         self.df.dropna(inplace=True)
        return self.df
    
    # ATR 
    def ATR(self, timeperiod):
        self.df['atr'] = talib.ATR(self.df['high'], self.df['low'], self.df['close'], timeperiod=timeperiod)
        return self.df
    
    # EMA 지수이동평균
    def EMA(self, timeperiod):
        self.df['ema'] = talib.EMA(self.df['close'], timeperiod=timeperiod)
        return self.df
    
    
# class Pivot_technical_indicators(Technical_indicators):
    
#     def __init__(self, ticker, interval, cnt):
#         self.df = df # 메인
#         self.df_ = df_ # 비교군

def find_target(df, df_, check):

    # 1h은 단순하게 스토케스틱이 20이하인 최신 그륩 바로 전 
    target_df = df.loc[check]

    # 15m봉과 merge
    df_merge = pd.merge(df_, target_df, how='inner', on=['date','hour'])

    # 15m에서 전 stoch rsi그룹의 가장 low값의 close, rsi 찾음
    return df_merge.loc[df_merge['low_x'] == df_merge['low_x'].min()][['time_x','date','hour','close_x', 'rsi_x']]

def check_regular_divergence(prev_price, prev_rsi, last_price, last_rsi):
    """
    레귤러, 히든 다이버전스 체크
    """
    last_price = last_price
    prev_price = prev_price
    last_rsi = last_rsi
    prev_rsi = prev_rsi

    if last_price < prev_price and last_rsi > prev_rsi:
        return 1, print('레귤러')
    elif last_price > prev_price and last_rsi < prev_rsi:
        return 1, print('히든')
    else:
        print(f'가격 : 최신 {last_price} vs 전 {prev_price}\n rsi : 최신 {last_rsi} vs 전 {prev_rsi}')


# 이 코드는 주어진 숫자 리스트를 역순으로 정렬한 후, 연속된 숫자들을 찾아 그룹화하는 함수를 정의합니다. 
# 그러나 두 번째 리스트까지만 그룹화하여 반환하는 기능이 추가되었습니다.
# group_consecutive_numbers_reverse 함수는 입력된 숫자 리스트를 역순으로 정렬합니다.
# 정렬된 리스트에서 각 숫자를 확인하면서 연속된 숫자인지 확인합니다.
# 현재 그룹에 숫자를 추가하거나 새로운 그룹을 시작합니다.
# 그룹이 완성되면 이를 결과 리스트에 추가합니다.
# 두 번째 리스트까지 그룹화된 후에는 계산을 중단하고 결과를 반환합니다.

def group_consecutive_numbers_reverse(numbers):
    groups = []
    current_group = []

    for number in sorted(numbers, reverse=True):
        if not current_group or number == current_group[-1] - 1:
            current_group.append(number)
        else:
            groups.append(current_group)
            current_group = [number]

        # 두 번째 리스트까지만 계산하고 종료
        if len(groups) == 2:
            break

    if current_group:
        groups.append(current_group)

    return groups

def fibonacci_retracement_sell_prices(high, low, levels):
    sell_prices = []

    for level in levels:
        sell_price = high - (level * (high - low))
        sell_prices.append((level, sell_price))
    return sell_prices

def zigzag(prices, up_thresh=0.05, down_thresh=-0.05):
    """
    ZigZag 알고리즘 (변곡점 감지)
    prices: list 또는 pandas.Series 형태의 가격 데이터
    up_thresh: 상승 전환 임계값 (예: 0.05 = 5%)
    down_thresh: 하락 전환 임계값 (예: -0.05 = -5%)
    """
    pivots = np.zeros(len(prices))
    last_pivot = prices[0]
    last_pivot_idx = 0
    trend = 0  # 0 = 초기, 1 = 상승, -1 = 하락

    for i in range(1, len(prices)):
        change = (prices[i] - last_pivot) / last_pivot

        if trend == 0:
            if change >= up_thresh:
                trend = 1
                last_pivot = prices[i]
                last_pivot_idx = i
                pivots[i] = 1  # peak
            elif change <= down_thresh:
                trend = -1
                last_pivot = prices[i]
                last_pivot_idx = i
                pivots[i] = -1  # valley

        elif trend == 1:
            if prices[i] > last_pivot:
                last_pivot = prices[i]
                last_pivot_idx = i
            elif change <= down_thresh:
                trend = -1
                pivots[last_pivot_idx] = 1
                last_pivot = prices[i]
                last_pivot_idx = i
                pivots[i] = -1

        elif trend == -1:
            if prices[i] < last_pivot:
                last_pivot = prices[i]
                last_pivot_idx = i
            elif change >= up_thresh:
                trend = 1
                pivots[last_pivot_idx] = -1
                last_pivot = prices[i]
                last_pivot_idx = i
                pivots[i] = 1

    return pivots


In [3]:
import time
from datetime import datetime
import pytz

import pandas as pd
import numpy as np
import talib
import requests
import json
from datetime import datetime, timedelta
import ccxt
from pytz import timezone
import sys
import json

api_key = 'Aq9SAIfjxtLpJFmfC6eXZHt7pVpka1zHR899oZ8kReBUmQiEwf46EdVCq0LtehLl'
api_secret = 'n1tOBlTdgVB0BTWfIrvHYGq3DXyPEPxVc2rudEX3C5Y6fHOC50rjZDvmHJU1Uq89'


exchange = ccxt.binance(config={
    'apiKey': api_key,
    'secret': api_secret,
    'enableRateLimit': True,
    'options': {
        'defaultType': 'future'
    }
})

ticker = 'BTC/USDT'
interval = '1d'
cnt=1000


C:\Users\hahonggu\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\hahonggu\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:61: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [4]:
# # 기술적지표 클래스 정의 
# tech_class = Technical_indicators(ticker=ticker, interval=interval, cnt=cnt)
# df = tech_class.data_select()

# # 기술적 지표들 넣기 
# tech_class.RSI(14)
# tech_class.EMA(100)
# tech_class.ATR(21)

# df['close_stoch_34'] = tech_class.STOCHRSI(34,3,0)['close_stoch']
# df['close_stoch_28'] = tech_class.STOCHRSI(28,3,0)['close_stoch']
# df.drop(['close_stoch'], axis=1,inplace=True)

In [5]:
df_1d = Technical_indicators(ticker=ticker, interval='1d', cnt=cnt)
df_4h = Technical_indicators(ticker=ticker, interval='4h', cnt=cnt)

df_1d.STOCHRSI(14,3,0)
df_1d.RSI(14)
df_1d.EMA(100)
df = df_1d.ATR(21)

df_4h.STOCHRSI(14,3,0)
df_4h.RSI(14)
df_4h.EMA(100)
df_31 = df_4h.ATR(21)

In [6]:
import numpy as np
import pandas as pd

def wilder_atr(high, low, close, period=14):
    high = np.asarray(high, dtype=float)
    low = np.asarray(low, dtype=float)
    close = np.asarray(close, dtype=float)
    prev_close = np.r_[close[0], close[:-1]]
    tr = np.maximum(high - low, np.maximum(np.abs(high - prev_close), np.abs(low - prev_close)))
    atr = pd.Series(tr).ewm(alpha=1/period, adjust=False).mean().values
    return atr

def zigzag_meaningful_v2(
    close, high=None, low=None,
    up_pct=0.05, down_pct=0.05,
    atr_period=14, atr_mult=2.0,
    threshold_mode='or',          # 'or' | 'and' | 'max'
    use_hl=True,
    min_bars=5,
    min_swing_atr=1.0,
    finalize_last=False,
    atr_arr=None 
):
    """
    개선된 의미형 ZigZag
    - 방향 전환 조건: 퍼센트 / ATR / 결합모드(or/and/max)
    - 피벗 극값은 High/Low 기준
    - 사후 가지치기: 교대성(peak↔valley) 보장, 스윙폭은 High/Low로 평가
    - 옵션: 마지막 피벗 강제 확정(finalize_last)
    """

        
    price = np.asarray(close, dtype=float)
    n = len(price)

    if use_hl:
        if high is None or low is None:
            raise ValueError("use_hl=True면 high/low가 필요합니다.")
        high = np.asarray(high, dtype=float)
        low  = np.asarray(low, dtype=float)
    else:
        high = price
        low  = price

    if atr_arr is None:
        atr = wilder_atr(high, low, price, period=atr_period)
    else:
        atr = np.asarray(atr_arr, dtype=float)

    def moved_up(ext_val, i):
        pct_ok = (high[i] - ext_val) / max(ext_val, 1e-12) >= up_pct
        atr_ok = (high[i] - ext_val) >= atr_mult * atr[i]
        if threshold_mode == 'and': return pct_ok and atr_ok
        if threshold_mode == 'max': return (high[i] - ext_val) >= max(up_pct*ext_val, atr_mult*atr[i])
        return pct_ok or atr_ok

    def moved_down(ext_val, i):
        pct_ok = (ext_val - low[i]) / max(ext_val, 1e-12) >= down_pct
        atr_ok = (ext_val - low[i]) >= atr_mult * atr[i]
        if threshold_mode == 'and': return pct_ok and atr_ok
        if threshold_mode == 'max': return (ext_val - low[i]) >= max(down_pct*ext_val, atr_mult*atr[i])
        return pct_ok or atr_ok

    pivots = np.zeros(n, dtype=int)
    trend = 0  # 0 초기, 1 상승 레그(고점 탐색), -1 하락 레그(저점 탐색)

    # 현재 진행 레그의 '극단값' (확정 전 후보)
    ext_idx = 0
    ext_val = price[0]

    # 초기 방향 결정
    for i in range(1, n):
        if trend == 0:
            if moved_up(ext_val, i):
                trend = 1
                ext_idx = i
                ext_val = high[i]
            elif moved_down(ext_val, i):
                trend = -1
                ext_idx = i
                ext_val = low[i]
            continue

        if trend == 1:
            # 상승 레그: 고점 갱신
            if high[i] > ext_val:
                ext_idx, ext_val = i, high[i]
            # 의미있는 하락 → 직전 고점 확정
            if moved_down(ext_val, i):
                pivots[ext_idx] = 1  # peak
                trend = -1
                ext_idx, ext_val = i, low[i]

        else:  # trend == -1
            # 하락 레그: 저점 갱신
            if low[i] < ext_val:
                ext_idx, ext_val = i, low[i]
            # 의미있는 상승 → 직전 저점 확정
            if moved_up(ext_val, i):
                pivots[ext_idx] = -1  # valley
                trend = 1
                ext_idx, ext_val = i, high[i]

    # 옵션: 마지막 미확정 피벗 확정
    if finalize_last and trend != 0:
        pivots[ext_idx] = 1 if trend == 1 else -1

    # === 사후 가지치기 ===
    def extreme_at(idx, sign):
        # sign: 1=peak → High, -1=valley → Low
        return high[idx] if sign == 1 else low[idx]

    idxs = np.where(pivots != 0)[0].tolist()

    # (A) 교대성 보장: 혹시라도 같은 부호가 연속되면 더 '극단'만 남김
    j = 1
    while j < len(idxs):
        a, b = idxs[j-1], idxs[j]
        if pivots[a] == pivots[b]:
            # 같은 부호 → 더 극단인 피벗만 유지
            if pivots[a] == 1:  # 두 고점 중 더 높은 것을 유지
                keep = a if high[a] >= high[b] else b
            else:               # 두 저점 중 더 낮은 것을 유지
                keep = a if low[a] <= low[b] else b
            drop = b if keep == a else a
            pivots[drop] = 0
            idxs.pop(j if drop == b else j-1)
            j = max(1, j-1)
        else:
            j += 1

    # (B) 약한 중간 피벗 제거(삼연속 a-b-c에서 b가 약하면 제거) + min_bars/폭 조건
    changed = True
    while changed and len(idxs) >= 3:
        changed = False
        k = 1
        while k < len(idxs)-1:
            a, b, c = idxs[k-1], idxs[k], idxs[k+1]
            sign_b = pivots[b]
            # 최소 봉 조건
            if (b - a) < min_bars or (c - b) < min_bars:
                pivots[b] = 0
                idxs.pop(k)
                changed = True
                continue
            # 스윙 폭(High/Low 기준) – b를 기준으로 양쪽 레그 최소폭 평가
            amp1 = abs(extreme_at(b, sign_b) - extreme_at(a, pivots[a]))
            amp2 = abs(extreme_at(c, pivots[c]) - extreme_at(b, sign_b))
            thr  = min_swing_atr * atr[b]
            if min(amp1, amp2) < thr:
                pivots[b] = 0
                idxs.pop(k)
                changed = True
                continue
            k += 1

    return pivots, atr



def latest_alternating_pivots(df, pivot_col='pivot', pivot_price_col='pivot_price'):
    """
    df[pivot_col] = 1(고점), -1(저점), 0(없음)
    최신 확정 피벗들 중 '부호가 다른' 두 개를 뒤에서부터 찾아 스윙 방향을 결정.
    - 저점(-1) → 고점(1) = 'up'
    - 고점(1) → 저점(-1) = 'down'
    반환: dict(방향/가격/인덱스)
    """
    pvals = df[pivot_col].to_numpy()
    idxs = np.flatnonzero(pvals != 0)
    if len(idxs) < 2:
        raise ValueError("확정 피벗이 2개 이상 필요합니다.")

    # 뒤에서부터 서로 부호 다른 쌍을 탐색
    for k in range(len(idxs)-1, 0, -1):
        i2 = idxs[k]
        i1 = idxs[k-1]
        s1, s2 = int(pvals[i1]), int(pvals[i2])
        if s1 == s2:
            continue
        p1 = float(df[pivot_price_col].iat[i1])
        p2 = float(df[pivot_price_col].iat[i2])

        if s1 == -1 and s2 == 1:
            direction = 'up'    # 저→고
        elif s1 == 1 and s2 == -1:
            direction = 'down'  # 고→저
        else:
            continue

        return {
            'start_pos': i1, 'end_pos': i2,             # 정수 포지션 인덱스
            'start_ts': df.index[i1], 'end_ts': df.index[i2],  # 라벨 인덱스(시간)
            'start_sign': s1, 'end_sign': s2,
            'start_price': p1, 'end_price': p2,
            'direction': direction
        }

    raise ValueError("교대(고↔저) 피벗 쌍을 찾을 수 없습니다. pruning/파라미터를 점검하세요.")


def fib_from_latest(df, pivot_col='pivot', pivot_price_col='pivot_price',
                    retracements=(0.236, 0.382, 0.5, 0.618, 0.786),
                    extensions=(1.272, 1.414, 1.618)):
    """
    최신 교대 피벗 쌍을 찾아:
    - 상승(저→고): 0=고점(hi), 1=저점(lo)
      level = hi - (hi - lo) * r
      ext   = hi + (hi - lo) * e
    - 하락(고→저): 0=저점(lo), 1=고점(hi)
      level = lo + (hi - lo) * r
      ext   = lo - (hi - lo) * e
    """
    swing = latest_alternating_pivots(df, pivot_col, pivot_price_col)

    if swing['direction'] == 'up':
        lo, hi = swing['start_price'], swing['end_price']
        anchors = {'0': hi, '1': lo}  # 차트에 그릴 때 0=끝(고점), 1=시작(저점)
        rets = {r: hi - (hi - lo) * r for r in retracements}
        exts = {e: hi + (hi - lo) * e for e in extensions}
        gp   = (hi - (hi - lo)*0.65, hi - (hi - lo)*0.618)  # 골든포켓(0.65~0.618)

    else:  # 'down'
        hi, lo = swing['start_price'], swing['end_price']
        anchors = {'0': lo, '1': hi}  # 0=끝(저점), 1=시작(고점)
        rets = {r: lo + (hi - lo) * r for r in retracements}
        exts = {e: lo - (hi - lo) * e for e in extensions}
        gp   = (lo + (hi - lo)*0.618, lo + (hi - lo)*0.65)

    gp = (min(gp), max(gp))
    return {
        'swing': swing,                 # 메타 정보
        'anchors': anchors,             # 0/1 앵커 값(차트 드로잉용)
        'retracements': rets,           # 되돌림 레벨
        'extensions': exts,             # 확장 레벨
        'golden_pocket': gp             # (low, high)
    }

def last_zigzag_swing(df, pivot_col='pivot', pivot_price_col='pivot_price'):
    """마지막 두 개 '확정' 피벗으로 스윙(방향/가격/인덱스) 추출"""
    idxs = df.index[df[pivot_col] != 0].tolist()
    if len(idxs) < 2:
        raise ValueError("확정 피벗이 2개 이상 필요합니다.")

    # 마지막 부호가 달라지는 지점(peak↔valley)로 보정
    i2 = idxs[-1]
    i1 = idxs[-2]
    if df.at[i1, pivot_col] == df.at[i2, pivot_col] and len(idxs) >= 3:
        i1 = idxs[-3]  # 교대가 안 되면 하나 더 과거로

    s1, s2 = int(df.at[i1, pivot_col]), int(df.at[i2, pivot_col])
    p1, p2 = float(df.at[i1, pivot_price_col]), float(df.at[i2, pivot_price_col])

    if s1 == -1 and s2 == 1:
        direction = 'up'    # 저점→고점
    elif s1 == 1 and s2 == -1:
        direction = 'down'  # 고점→저점
    else:
        # 혹시 같은 부호가 연속되면 더 과거 피벗을 탐색
        raise ValueError("마지막 두 피벗이 교대 상태가 아닙니다. pruning 설정을 확인하세요.")

    return {
        'start_ts': i1, 'end_ts': i2,
        'start_sign': s1, 'end_sign': s2,
        'start_price': p1, 'end_price': p2,
        'direction': direction
    }

def fib_ratio_cluster_zones(df, res,
                            atr_col='atr',
                            use=('retracements',),     # ('retracements','extensions') 도 가능
                            k=0.5,                     # 기본 승수
                            mode='linear',             # 'linear' => g(r)=r, 'sqrt' => g(r)=sqrt(r), 'pow' => r**gamma
                            gamma=1.0,                 # mode='pow'일 때 지수
                            max_half=None,             # 반폭 상한 (예: 1.2*ATR)
                            merge_gap_mult=0.0,        # 존 사이 간격이 <= 이 값*ATR이면 병합
                            sort_by='strength'):       # 'strength' or 'center'
    end_pos = res['swing']['end_pos']
    if atr_col not in df.columns:
        raise ValueError(f"df['{atr_col}']가 필요합니다.")
    # 끝 피벗 시점 직전까지의 유효 ATR 사용(초기 NaN 대비)
    local_atr = float(pd.Series(df[atr_col]).iloc[:end_pos+1].dropna().iloc[-1])

    # 사용할 레벨들 수집
    lvls = []
    if 'retracements' in use:
        for r, v in res['retracements'].items():
            lvls.append({'ratio': r, 'label': f'R{int(r*100)}', 'price': float(v)})
    if 'extensions' in use:
        for e, v in res['extensions'].items():
            lvls.append({'ratio': float(e), 'label': f'E{str(e)}', 'price': float(v)})

    def g(r):
        if mode == 'sqrt': return np.sqrt(r)
        if mode == 'pow' : return r ** gamma
        return r  # linear

    intervals = []
    for L in lvls:
        half = k * g(L['ratio']) * local_atr
        if max_half is not None:
            half = min(half, max_half)
        intervals.append({
            'low': L['price'] - half,
            'high': L['price'] + half,
            'labels': [L['label']],
            'levels': [L],
            'ratio': L['ratio']
        })

    # 정렬/병합
    intervals.sort(key=lambda x: x['low'])
    zones = []
    for seg in intervals:
        if not zones:
            zones.append(seg); continue
        prev = zones[-1]
        overlap = seg['low'] <= prev['high']
        gap_small = (seg['low'] - prev['high']) <= (merge_gap_mult * local_atr)
        if overlap or gap_small:
            prev['high'] = max(prev['high'], seg['high'])
            prev['labels'].extend(seg['labels'])
            prev['levels'].extend(seg['levels'])
        else:
            zones.append(seg)

    # 메타데이터
    for z in zones:
        z['center'] = 0.5 * (z['low'] + z['high'])
        z['strength'] = len(z['labels'])
        z['labels'] = sorted(set(z['labels']), key=lambda s: (s[0], s[1:]))

    if sort_by == 'strength':
        zones.sort(key=lambda z: (-z['strength'], z['center']))
    elif sort_by == 'center':
        zones.sort(key=lambda z: z['center'])

    # 레벨→존 매핑
    levels = []
    for i, z in enumerate(zones):
        for L in z['levels']:
            levels.append({'label': L['label'], 'price': L['price'], 'zone_idx': i})

    return zones, levels, local_atr

# 피보나치 바운더리 구하기

In [9]:
# ❶ 파라미터 프리셋(권장)
# - 코인 15m~1H: threshold_mode='max', up/down=0.04~0.06, atr_mult=1.8~2.5, min_bars=5~12
# - 주식 일봉: threshold_mode='and', up/down=0.05~0.08, atr_mult=1.5~3.0, min_bars=5~10
pivots, _ = zigzag_meaningful_v2(
    close=df['close'], high=df['high'], low=df['low'],
    up_pct=0.04, down_pct=0.04,
    atr_period=21, atr_mult=1.8,
    threshold_mode='max',
    use_hl=True,
    min_bars=7,
    min_swing_atr=1.0,
    finalize_last=False  # 실시간이면 False, 백테스트/리포트면 True
)

# ❷ 결과를 df에 붙이기
df['pivot'] = pivots                    # 1=고점, -1=저점
df['pivot_price'] = np.where(
    df['pivot']==1, df['high'],
    np.where(df['pivot']==-1, df['low'], np.nan)
)

# df['atr'] = atr   # ←←← 이 줄이 반드시 필요!
# 1) 마지막 스윙 추출
swing = last_zigzag_swing(df, pivot_col='pivot', pivot_price_col='pivot_price')

# 2) 피보나치 계산
# 지그재그 계산 끝난 df에 pivot/pivot_price가 붙어있다고 가정
res = fib_from_latest(df, pivot_col='pivot', pivot_price_col='pivot_price')

print("swing:", res['swing']['direction'], res['swing']['start_ts'], "→", res['swing']['end_ts'])
print("anchors(0/1):", res['anchors'])
print("retracements:", {int(k*100): round(v, 2) for k, v in res['retracements'].items()})
print("extensions:", {k: round(v, 2) for k, v in res['extensions'].items()})
print("golden pocket:", tuple(round(x,2) for x in res['golden_pocket']))


dir_kor = "상승(저→고)" if res['swing']['direction'] == 'up' else "하락(고→저)"
print(f"[ZigZag] 현재 방향: {dir_kor}")
print(f"시작 피벗: {res['swing']['start_ts']}  "
      f"가격: {res['swing']['start_price']:.2f}  "
      f"타입: {'저점' if res['swing']['start_sign']==-1 else '고점'}")
print(f"끝   피벗: {res['swing']['end_ts']}  "
      f"가격: {res['swing']['end_price']:.2f}  "
      f"타입: {'고점' if res['swing']['end_sign']==1 else '저점'}")

# **상승(저→고)**일 때 0 = 고점, 1 = 저점 → 즉 1 → 0 입니다.

# **하락(고→저)**일 때 1 = 고점, 0 = 저점 → 즉 0 → 1 입니다.
_, _, atr_end = fib_ratio_cluster_zones(
    df, res, atr_col='atr',
    use=('retracements',),
    k=1.0, mode='linear',  # 아무 값 ok. 여기서는 atr_end만 쓰려는 목적
    merge_gap_mult=0.25
)

def print_per_ratio_boundaries(df, res, atr_col='atr',
                               ratios=(0.236, 0.382, 0.5, 0.618, 0.786),
                               k=1.0, mode='linear', gamma=1.0):
    # 끝 피벗 시점의 유효 ATR
    end_pos = res['swing']['end_pos']
    atr_end = float(pd.Series(df[atr_col]).iloc[:end_pos+1].dropna().iloc[-1])

    def g(r):
        if mode == 'sqrt': return np.sqrt(r)
        if mode == 'pow' : return r**gamma
        return r  # linear

    for r in ratios:
        lvl = res['retracements'][r]    # 피보나치 레벨 값
        half = k * g(r) * atr_end       # 반폭 = k*g(r)*ATR
        low, high = lvl - half, lvl + half
        print(f"R{int(r*100):>2}: {low:.6f} ~ {high:.6f}")

# 사용 예: (r×ATR 그대로)
print_per_ratio_boundaries(df, res, atr_col='atr', k=1.0, mode='linear')
    

swing: up 986 → 996
anchors(0/1): {'0': 126208.5, '1': 108566.0}
retracements: {23: 122044.87, 38: 119469.07, 50: 117387.25, 61: 115305.43, 78: 112341.49}
extensions: {1.272: 148649.76, 1.414: 151154.99, 1.618: 154754.07}
golden pocket: (114740.88, 115305.43)
[ZigZag] 현재 방향: 상승(저→고)
시작 피벗: 986  가격: 108566.00  타입: 저점
끝   피벗: 996  가격: 126208.50  타입: 고점
R23: 121406.058181 ~ 122683.681819
R38: 118435.056039 ~ 120503.073961
R50: 116033.835130 ~ 118740.664870
R61: 113632.614221 ~ 116978.255779
R78: 110213.926824 ~ 114469.063176


# STOCH RSI + RSI 바운더리 구하기

In [17]:
import numpy as np
import pandas as pd
import talib

# ─────────────────────────────────────────────
# (이미 가지고 있는 함수 그대로 사용 가능)
def decompose_oversold_segments_D(df, d_col='close_stoch', oversold=20, auto_scale=True):
    if d_col not in df.columns:
        raise ValueError(f"{d_col} 컬럼이 없습니다.")
    s = df[d_col].to_numpy(dtype=float)
    smax = np.nanmax(s)
    thr = oversold/100.0 if (auto_scale and smax <= 1.5) else float(oversold)

    segs = []
    i = len(s)-1
    while i >= 0:
        v = s[i]
        if np.isfinite(v) and v <= thr:
            b = i; a = i
            while a-1 >= 0 and np.isfinite(s[a-1]) and s[a-1] <= thr:
                a -= 1
            segs.append((a,b))
            i = a-1
        else:
            i -= 1
    segs.reverse()
    return segs, thr

def ref_from_segment_min_close(df, seg, rsi_col='rsi'):
    a, b = seg
    sub = df.iloc[a:b+1]
    idx_min = sub['close'].idxmin()
    return {
        'ref_idx': int(df.index.get_loc(idx_min)),
        'ref_ts': idx_min,
        'ref_price': float(df.at[idx_min, 'close']),
        'ref_rsi': float(df.at[idx_min, rsi_col]),
        'seg_start': a, 'seg_end': b
    }
# ─────────────────────────────────────────────


# ✅ 핵심 1) 세그먼트 선택자: "현재 D≤20이면 직전 세그먼트 강제"
def pick_oversold_segment_D_with_current_rule(
    df, *, d_col='close_stoch', oversold=20, auto_scale=True,
    fallback_mode='latest'  # 'latest' 또는 'prev' (직전이 없을 때의 대체 동작)
):
    """
    규칙 A: 현재 close_stoch ≤ oversold → 최신 세그먼트를 건너뛰고 '직전' 세그먼트를 선택
            (직전이 없으면 None 반환; fallback_mode로 대체 선택 가능)
    규칙 B: 현재 close_stoch > oversold → 가장 최근의 D≤20 세그먼트(과거의 최신 세그먼트)를 선택
    반환: (seg or None, thr_used, reason)
    """
    segs, thr = decompose_oversold_segments_D(df, d_col=d_col, oversold=oversold, auto_scale=auto_scale)
    if not segs:
        return None, thr, "no_segment"

    cur_d = float(df[d_col].iloc[-1])
    n = len(df)

    # 최신 세그먼트
    last_seg = segs[-1]
    a, b = last_seg

    if cur_d <= thr:
        # 현재 봉이 오버솔드 → 최신 세그먼트는 반드시 끝이 현재 근처(b==n-1)일 가능성 큼
        # 요구사항: "현재가 오버솔드면 '직전' 세그먼트를 써라"
        if len(segs) >= 2:
            return segs[-2], thr, "current_oversold_use_prev"
        else:
            # 직전이 없으면 fallback
            if fallback_mode == 'latest':
                return last_seg, thr, "current_oversold_no_prev_fallback_latest"
            elif fallback_mode == 'prev':
                return None, thr, "current_oversold_no_prev_none"
            else:
                return last_seg, thr, "current_oversold_no_prev_fallback_latest"
    else:
        # 현재 오버솔드가 아님 → 가장 최근에 있었던 오버솔드 세그먼트를 그대로 사용
        return last_seg, thr, "current_not_oversold_use_latest"


# ✅ 핵심 2) 리포트: 위 규칙을 적용해 ref 잡고, 다이버전스 필요/가능 가격 출력
def needed_close_regular_now(df, *, ref_price, ref_rsi, rsi_period=14,
                             lower_bound=None, eps=1e-8, tol=1e-6, max_iter=60):
    close = df['close'].to_numpy(dtype=float).copy()
    def rsi_last(x):
        close[-1] = x
        return talib.RSI(close, timeperiod=rsi_period)[-1]

    U = ref_price - max(eps, abs(ref_price)*1e-6)  # ref보다 낮아야
    L = U - max(1e-6, abs(U)*0.10) if lower_bound is None else min(lower_bound, U - max(1e-6, abs(U)*0.001))
    if not np.isfinite(U) or L >= U:
        return None
    rU = rsi_last(U)
    if (not np.isfinite(rU)) or rU <= ref_rsi:
        return None

    lo, hi = L, U
    for _ in range(max_iter):
        mid = (lo+hi)/2.0
        rmid = rsi_last(mid)
        if not np.isfinite(rmid):
            lo = mid; continue
        if rmid > ref_rsi: hi = mid
        else:              lo = mid
        if abs(hi-lo) <= tol: break
    return float(min(hi, U))

def feasible_close_range_hidden_now(df, *, ref_price, ref_rsi, rsi_period=14,
                                    upper_bound=None, eps=1e-8, tol=1e-6, max_iter=60):
    close = df['close'].to_numpy(dtype=float).copy()
    def rsi_last(x):
        close[-1] = x
        return talib.RSI(close, timeperiod=rsi_period)[-1]

    L = ref_price + max(eps, abs(ref_price)*1e-6)  # ref보다 높아야
    U = L + max(1e-6, abs(L)*0.10) if upper_bound is None else max(upper_bound, L + max(1e-6, abs(L)*0.001))
    if not np.isfinite(L) or L >= U:
        return None
    rL = rsi_last(L)
    if (not np.isfinite(rL)) or rL >= ref_rsi:
        return None

    lo, hi = L, U
    for _ in range(max_iter):
        mid = (lo+hi)/2.0
        rmid = rsi_last(mid)
        if not np.isfinite(rmid):
            hi = mid; continue
        if rmid < ref_rsi: lo = mid
        else:              hi = mid
        if abs(hi-lo) <= tol: break
    xmax = float(lo)
    if xmax <= L: return None
    return (float(L), xmax)

def print_divergence_report_D_current_rule(
    df, *, rsi_col='rsi', d_col='close_stoch', rsi_period=14,
    oversold=20, auto_scale=True, fallback_mode='latest'
):
    """
    - 현재 봉이 오버솔드(≤20)면 '직전' 오버솔드 세그먼트 사용
    - 아니면 가장 최근의 오버솔드 세그먼트 사용
    """
    if 'close' not in df.columns: raise ValueError("close 컬럼 없음")
    if rsi_col not in df.columns: raise ValueError(f"{rsi_col} 컬럼 없음")
    if d_col   not in df.columns: raise ValueError(f"{d_col} 컬럼 없음")

    seg, thr_used, reason = pick_oversold_segment_D_with_current_rule(
        df, d_col=d_col, oversold=oversold, auto_scale=auto_scale, fallback_mode=fallback_mode
    )
    if seg is None:
        print(f"오버솔드 세그먼트 선택 실패 (reason={reason}, thr_used={thr_used})")
        return

    ref = ref_from_segment_min_close(df, seg, rsi_col=rsi_col)

    price_now = float(df['close'].iloc[-1])
    rsi_now   = float(df[rsi_col].iloc[-1])

    need_reg = needed_close_regular_now(
        df, ref_price=ref['ref_price'], ref_rsi=ref['ref_rsi'],
        rsi_period=rsi_period
    )
    feas_hid = feasible_close_range_hidden_now(
        df, ref_price=ref['ref_price'], ref_rsi=ref['ref_rsi'],
        rsi_period=rsi_period
    )

    ts = df['datetime'] if 'datetime' in df.columns else df.index
    a,b = seg
    print("\n=== Divergence Check (current-rule) ===")
    print(f"thr_used={thr_used} | reason={reason}")
    print(f"segment: [{a} ~ {b}]  ({ts.iloc[a]} → {ts.iloc[b]})")
    print(f"ref(min close): ts={ref['ref_ts']}  price={ref['ref_price']:.6f}  rsi={ref['ref_rsi']:.3f}")
    print(f"now           : price={price_now:.6f}  rsi={rsi_now:.3f}")

    regular_now = (price_now < ref['ref_price']) and (rsi_now > ref['ref_rsi'])
    hidden_now  = (price_now > ref['ref_price']) and (rsi_now < ref['ref_rsi'])

    print("\n[Regular Bullish] price<ref & RSI>ref")
    print(f" - 현재 성립?  -> {regular_now}")
    if need_reg is None: print(" - 현재봉 성립 최소 종가: 불가")
    else:                print(f" - 현재봉 성립 최소 종가: {need_reg:.6f}")

    print("\n[Hidden Bullish]  price>ref & RSI<ref")
    print(f" - 현재 성립?  -> {hidden_now}")
    if feas_hid is None: print(" - 현재봉에서 가능한 종가 범위: 없음")
    else:
        L, Xmax = feas_hid
        inside = (L <= price_now <= Xmax)
        print(f" - 가능한 종가 범위: {L:.6f} ~ {Xmax:.6f}   (현재가 in-range? {inside})")


In [18]:
print_divergence_report_D_current_rule(
    df,
    rsi_col='rsi', d_col='close_stoch',
    oversold=20, auto_scale=True,
    fallback_mode='latest'   # 현재도 오버솔드인데 직전 세그먼트가 없으면 최신으로 fallback
)



=== Divergence Check (current-rule) ===
thr_used=20.0 | reason=current_oversold_use_prev
segment: [959 ~ 961]  (2025-08-30 09:00:00 → 2025-09-01 09:00:00)
ref(min close): ts=960  price=108208.400000  rsi=37.840
now           : price=108934.500000  rsi=35.980

[Regular Bullish] price<ref & RSI>ref
 - 현재 성립?  -> False
 - 현재봉 성립 최소 종가: 불가

[Hidden Bullish]  price>ref & RSI<ref
 - 현재 성립?  -> True
 - 가능한 종가 범위: 108208.508208 ~ 109819.894899   (현재가 in-range? True)


# 피보나치 + RSI 바운더리 교집합 구하기

In [29]:
# === 1) 세그먼트 선택: 현재봉이 오버솔드(≤20)면 '직전' 세그먼트를, 아니면 '최신' 세그먼트를 사용 ===
import numpy as np
import talib

def _stoch_d_threshold(series, oversold=20, auto_scale=True):
    smax = np.nanmax(series.astype(float).to_numpy())
    return oversold/100.0 if (auto_scale and smax <= 1.5) else float(oversold)

def decompose_oversold_segments_D(df, d_col='close_stoch', oversold=20, auto_scale=True):
    thr = _stoch_d_threshold(df[d_col], oversold, auto_scale)
    d = df[d_col].astype(float).to_numpy()
    segs = []
    i = len(d) - 1
    while i >= 0:
        v = d[i]
        if np.isfinite(v) and v <= thr:
            b = i; a = i
            while a-1 >= 0 and np.isfinite(d[a-1]) and d[a-1] <= thr:
                a -= 1
            segs.append((a, b))
            i = a - 1
        else:
            i -= 1
    segs.reverse()
    return segs, thr

def pick_oversold_segment_D_with_current_rule(df, d_col='close_stoch', oversold=20, auto_scale=True):
    segs, thr = decompose_oversold_segments_D(df, d_col=d_col, oversold=oversold, auto_scale=auto_scale)
    if not segs:
        return None, thr, "no_segment"
    cur_d = float(df[d_col].iloc[-1])
    # 현재 오버솔드면 '직전', 아니면 '최신'
    if cur_d <= thr:
        return (segs[-2], thr, "current_oversold_use_prev") if len(segs) >= 2 else (segs[-1], thr, "fallback_latest")
    return segs[-1], thr, "current_not_oversold_use_latest"

def ref_from_segment_min_close(df, seg, rsi_col='rsi'):
    a, b = seg
    sub = df.iloc[a:b+1]
    idx_min = sub['close'].idxmin()
    return {
        'ref_idx': int(df.index.get_loc(idx_min)),
        'ref_ts': idx_min,
        'ref_price': float(df.at[idx_min, 'close']),
        'ref_rsi': float(df.at[idx_min, rsi_col]),
        'seg_start': a, 'seg_end': b
    }

# === 2) 다이버전스 만족을 위한 현재봉 '필요/가능' 가격 계산 ===
def needed_close_regular_now(df, *, ref_price, ref_rsi, rsi_period=14,
                             lower_bound=None, eps=1e-8, tol=1e-6, max_iter=60):
    close = df['close'].astype(float).to_numpy().copy()
    def rsi_last(x):
        close[-1] = x
        return talib.RSI(close, timeperiod=rsi_period)[-1]
    U = ref_price - max(eps, abs(ref_price)*1e-6)
    L = U - max(1e-6, abs(U)*0.10) if lower_bound is None else min(lower_bound, U - max(1e-6, abs(U)*0.001))
    if not np.isfinite(U) or L >= U: return None
    rU = rsi_last(U)
    if (not np.isfinite(rU)) or rU <= ref_rsi: return None
    lo, hi = L, U
    for _ in range(max_iter):
        mid = (lo+hi)/2.0
        rmid = rsi_last(mid)
        if not np.isfinite(rmid): lo = mid; continue
        if rmid > ref_rsi: hi = mid
        else:              lo = mid
        if abs(hi-lo) <= tol: break
    return float(min(hi, U))

def feasible_close_range_hidden_now(df, *, ref_price, ref_rsi, rsi_period=14,
                                    upper_bound=None, eps=1e-8, tol=1e-6, max_iter=60):
    close = df['close'].astype(float).to_numpy().copy()
    def rsi_last(x):
        close[-1] = x
        return talib.RSI(close, timeperiod=rsi_period)[-1]
    L = ref_price + max(eps, abs(ref_price)*1e-6)
    U = L + max(1e-6, abs(L)*0.10) if upper_bound is None else max(upper_bound, L + max(1e-6, abs(L)*0.001))
    if not np.isfinite(L) or L >= U: return None
    rL = rsi_last(L)
    if (not np.isfinite(rL)) or rL >= ref_rsi: return None
    lo, hi = L, U
    for _ in range(max_iter):
        mid = (lo+hi)/2.0
        rmid = rsi_last(mid)
        if not np.isfinite(rmid): hi = mid; continue
        if rmid < ref_rsi: lo = mid
        else:              hi = mid
        if abs(hi-lo) <= tol: break
    xmax = float(lo)
    return None if xmax <= L else (float(L), xmax)

# (1) RSI 목표값을 만족하는 '현재봉 종가'를 바운더리 [L,H] 안에서 역산
def _price_for_rsi_in_range(df, target_rsi, L, H, rsi_period=14, tol=1e-6, max_iter=60):
    """
    현재봉이 L~H 사이로 마감한다고 가정할 때 RSI가 target_rsi가 되도록 하는 가격을
    이 구간 내부에서 이분법으로 찾는다. bracket이 없으면 None 반환.
    """
    import numpy as np
    def f(p):
        return _rsi_at_price(df, p, rsi_period) - float(target_rsi)

    L = float(L); H = float(H)
    if not np.isfinite(L) or not np.isfinite(H) or H <= L:
        return None

    fL = f(L); fH = f(H)
    if not np.isfinite(fL) or not np.isfinite(fH):
        return None
    if abs(fL) <= tol: return float(L)
    if abs(fH) <= tol: return float(H)
    # bracket 검사(부호가 달라야 루트가 확실)
    if fL * fH > 0:
        return None

    lo, hi = L, H
    for _ in range(max_iter):
        mid = (lo + hi) / 2.0
        fm  = f(mid)
        if not np.isfinite(fm):
            # 수치 불안정 시 살짝 축소
            hi = mid
            continue
        if abs(fm) <= tol:
            return float(mid)
        if fL * fm <= 0:
            hi = mid; fH = fm
        else:
            lo = mid; fL = fm
        if abs(hi - lo) <= max(tol, 1e-9 * max(1.0, abs(L), abs(H))):
            break
    return float((lo + hi) / 2.0)

# (2) 출력 함수: 레벨 라인에 "RSI→PRICE"도 함께 표기 (옵션 rsi_marks로 특정 RSI들도 역산)
def print_level_divergence_hits(df, bounds, *, need_reg=None, hid_range=None, rsi_period=14,
                                rsi_marks=None):
    """
    bounds: build_fib_ratio_boundaries(...) 결과
    need_reg: needed_close_regular_now(...) (float 또는 None)
    hid_range: feasible_close_range_hidden_now(...)  (L, Xmax) 또는 None
    rsi_marks: 예) (30, 40, 50, 60) 같이 특정 RSI값에 대해 해당 레벨 바운더리 안의 가격을 역산해 같이 출력
    """
    import numpy as np

    # 유효성 보정
    if need_reg is not None:
        try: need_reg = float(need_reg)
        except: need_reg = None
    if hid_range is not None:
        try:
            L, X = float(hid_range[0]), float(hid_range[1])
            if not (np.isfinite(L) and np.isfinite(X)): hid_range = None
            else: hid_range = (L, X)
        except: hid_range = None

    print("\n=== LEVEL × DIVERGENCE CHECK ===")
    for b in bounds:
        lbl, low, high = b['label'], float(b['low']), float(b['high'])

        # 경계 가격에서의 RSI (현재 봉이 그 가격으로 마감한다고 가정)
        rsi_lo = _rsi_at_price(df, low,  rsi_period)
        rsi_hi = _rsi_at_price(df, high, rsi_period)

        # Regular: 가격∈[low, min(high, need_reg)]
        reg_hit, reg_rng = False, None
        if need_reg is not None and np.isfinite(need_reg):
            hi = min(high, need_reg)
            if hi > low:
                reg_hit, reg_rng = True, (low, hi)

        # Hidden: 가격∈[max(low,L), min(high,Xmax)]
        hid_hit, hid_rng = False, None
        if hid_range is not None:
            Lr, Xmax = hid_range
            lo = max(low, Lr); hi = min(high, Xmax)
            if hi > lo:
                hid_hit, hid_rng = True, (lo, hi)

        # 둘 다 겹치면 교집합(최고 컨플루언스)
        both_rng = None
        if reg_rng and hid_rng:
            lo = max(reg_rng[0], hid_rng[0]); hi = min(reg_rng[1], hid_rng[1])
            if hi > lo: both_rng = (lo, hi)

        # ── 추가: RSI→PRICE (기본은 경계 RSI에 대한 역산, 선택적으로 rsi_marks도 역산)
        # 경계 RSI에 대한 역산(검산 겸 표시)
        inv_lo = _price_for_rsi_in_range(df, rsi_lo, low, high, rsi_period)
        inv_hi = _price_for_rsi_in_range(df, rsi_hi, low, high, rsi_period)

        print(f"{lbl}: Price[{low:,.2f} ~ {high:,.2f}] | RSI[{rsi_lo:.2f} ~ {rsi_hi:.2f}]")
        if inv_lo is not None and inv_hi is not None:
            print(f"   · RSI→Price (boundary): {rsi_lo:.2f}→{inv_lo:,.2f} , {rsi_hi:.2f}→{inv_hi:,.2f}")
        else:
            print(f"   · RSI→Price (boundary): -")

        # 선택: 임의의 RSI 마크들에 대한 역산도 추가
        if rsi_marks:
            pairs = []
            for m in sorted(rsi_marks):
                p = _price_for_rsi_in_range(df, m, low, high, rsi_period)
                if p is not None:
                    pairs.append(f"{float(m):.2f}→{p:,.2f}")
            print("   · RSI→Price (marks): " + (", ".join(pairs) if pairs else "(none)"))

        print("   - Regular :", "YES" if reg_hit else "no",
              (f"  overlap={reg_rng[0]:,.2f}~{reg_rng[1]:,.2f}" if reg_rng else ""))
        print("   - Hidden  :", "YES" if hid_hit else "no",
              (f"  overlap={hid_rng[0]:,.2f}~{hid_rng[1]:,.2f}" if hid_rng else ""))
        if both_rng:
            print(f"   - BOTH    : {both_rng[0]:,.2f} ~ {both_rng[1]:,.2f}  (최고 컨플루언스)")

            
# === 3) ref 계산 → need_reg / hid_range 생성 ===
seg, thr_used, reason = pick_oversold_segment_D_with_current_rule(df, d_col='close_stoch', oversold=20, auto_scale=True)
if seg is None:
    raise RuntimeError("오버솔드 세그먼트를 찾지 못했습니다. (close_stoch 컬럼/스케일/빈값 확인)")

ref = ref_from_segment_min_close(df, seg, rsi_col='rsi')

need_reg = needed_close_regular_now(df, ref_price=ref['ref_price'], ref_rsi=ref['ref_rsi'], rsi_period=14)
hid_range = feasible_close_range_hidden_now(df, ref_price=ref['ref_price'], ref_rsi=ref['ref_rsi'], rsi_period=14)

print(f"[DEBUG] seg={seg}, thr_used={thr_used}, reason={reason}")
print(f"[DEBUG] ref_price={ref['ref_price']:.6f}, ref_rsi={ref['ref_rsi']:.3f}")
print(f"[DEBUG] need_reg={need_reg}, hid_range={hid_range}")


[DEBUG] seg=(959, 961), thr_used=20.0, reason=current_oversold_use_prev
[DEBUG] ref_price=108208.400000, ref_rsi=37.840
[DEBUG] need_reg=None, hid_range=(108208.50820839999, 109819.89489941855)


In [30]:
bounds, atr_end = build_fib_ratio_boundaries(
    df, res, atr_col='atr',
    ratios=(0.786, 0.618, 0.5, 0.382, 0.236),
    k=1.0, mode='linear', max_half_mult=1.2
)

# need_reg, hid_range 는 앞에서 계산한 값 사용
print_level_divergence_hits(
    df, bounds,
    need_reg=need_reg, hid_range=hid_range, rsi_period=14,
    rsi_marks=(30, 40, 50, 60)   # <- 원하면 표시. 생략 가능
)



=== LEVEL × DIVERGENCE CHECK ===
R79: Price[110,325.93 ~ 114,357.06] | RSI[38.99 ~ 51.26]
   · RSI→Price (boundary): 38.99→110,325.93 , 51.26→114,357.06
   · RSI→Price (marks): 40.00→110,744.54, 50.00→113,984.23
   - Regular : no 
   - Hidden  : no 
R62: Price[113,720.68 ~ 116,890.19] | RSI[49.07 ~ 58.39]
   · RSI→Price (boundary): 49.07→113,720.68 , 58.39→116,890.19
   · RSI→Price (marks): 50.00→113,984.23
   - Regular : no 
   - Hidden  : no 
R50: Price[116,105.08 ~ 118,669.42] | RSI[56.41 ~ 62.26]
   · RSI→Price (boundary): 56.41→116,105.08 , 62.26→118,669.42
   · RSI→Price (marks): 60.00→117,589.03
   - Regular : no 
   - Hidden  : no 
R38: Price[118,489.49 ~ 120,448.64] | RSI[61.90 ~ 65.48]
   · RSI→Price (boundary): 61.90→118,489.49 , 65.48→120,448.64
   · RSI→Price (marks): (none)
   - Regular : no 
   - Hidden  : no 
R24: Price[121,439.69 ~ 122,650.05] | RSI[67.04 ~ 68.77]
   · RSI→Price (boundary): 67.04→121,439.69 , 68.77→122,650.05
   · RSI→Price (marks): (none)
   - Regula

In [25]:
# import numpy as np

# # ─────────────────────────────────────────────
# # 1) RSI(target) → price 역산 (바운더리 [low, high] 내부에서만)
# # ─────────────────────────────────────────────
# def price_at_rsi_in_zone(df, rsi_target, low, high, rsi_period=14, tol=1e-6, max_iter=60):
#     """
#     바운더리 [low, high] 내에서 '이 봉이 rsi_target으로 마감'하려면 필요한 종가를 이분탐색으로 추정.
#     - 브라켓(저/고 경계의 RSI)이 타깃을 감싸지 않으면 None.
#     """
#     r_low = _rsi_at_price(df, low,  rsi_period)
#     r_high= _rsi_at_price(df, high, rsi_period)

#     # 오름차순/내림차순 모두 대응
#     asc = r_low <= r_high
#     r_min, r_max = (r_low, r_high) if asc else (r_high, r_low)
#     if not (r_min <= rsi_target <= r_max):
#         return None  # 타깃 RSI가 이 존의 RSI 범위를 벗어남

#     lo, hi = (low, high) if asc else (high, low)
#     for _ in range(max_iter):
#         mid = 0.5*(lo+hi)
#         r_mid = _rsi_at_price(df, mid, rsi_period)
#         if (asc and r_mid < rsi_target) or ((not asc) and r_mid > rsi_target):
#             lo = mid
#         else:
#             hi = mid
#         if abs(hi-lo) <= tol:
#             break
#     price = 0.5*(lo+hi)
#     return float(price)

# # ─────────────────────────────────────────────
# # 2) RSI 구간 [a,b] → 가격 구간 [P(a), P(b)] (교집합만)
# # ─────────────────────────────────────────────
# def price_band_for_rsi_band(df, rsi_a, rsi_b, low, high, rsi_period=14):
#     """
#     RSI 밴드 [rsi_a, rsi_b]가 존의 RSI 범위와 겹치면 가격 밴드 반환, 아니면 None.
#     """
#     r_low = _rsi_at_price(df, low,  rsi_period)
#     r_high= _rsi_at_price(df, high, rsi_period)
#     r_lo_band, r_hi_band = min(rsi_a, rsi_b), max(rsi_a, rsi_b)
#     r_zone_lo, r_zone_hi = min(r_low, r_high), max(r_low, r_high)

#     # 교집합
#     r_int_lo = max(r_lo_band, r_zone_lo)
#     r_int_hi = min(r_hi_band, r_zone_hi)
#     if r_int_hi <= r_int_lo:
#         return None

#     p1 = price_at_rsi_in_zone(df, r_int_lo, low, high, rsi_period=rsi_period)
#     p2 = price_at_rsi_in_zone(df, r_int_hi, low, high, rsi_period=rsi_period)
#     if (p1 is None) or (p2 is None):
#         return None
#     return (min(p1, p2), max(p1, p2))

# # ─────────────────────────────────────────────
# # 3) 레벨별로 Price/RSI 바운더리 + RSI→가격 역산 바운더리 출력
# # ─────────────────────────────────────────────
# def print_fib_price_and_rsi_inversions(df, res, *,
#                                        atr_col='atr',
#                                        ratios=(0.786,0.618,0.5,0.382,0.236),  # R79,R62,R50,R38,R24
#                                        k=1.0, mode='linear', max_half_mult=1.2,
#                                        rsi_period=14,
#                                        rsi_bands=((20,30),(30,40),(40,50),(50,60),(60,70)),
#                                        rsi_targets=None):
#     """
#     - 각 레벨의 Price zone, RSI zone 출력
#     - rsi_bands 각각에 대해 '가격 바운더리' 역산해 출력 (존 RSI와 겹칠 때만)
#     - rsi_targets(단일 RSI 값들)이 주어지면 그때의 가격도 역산해 출력
#     """
#     bounds, atr_end = build_fib_ratio_boundaries(
#         df, res, atr_col=atr_col, ratios=ratios,
#         k=k, mode=mode, max_half_mult=max_half_mult
#     )
#     print(f"\n=== FIB PRICE & RSI & (RSI→PRICE) BOUNDARIES ===")
#     print(f"Swing: {res['swing']['direction']} | ATR(end)={atr_end:.6f} | k={k}, mode={mode}")

#     for b in bounds:
#         lbl, low, high, center, half = b['label'], b['low'], b['high'], b['level'], b['half']
#         rsi_lo = _rsi_at_price(df, low,  rsi_period)
#         rsi_hi = _rsi_at_price(df, high, rsi_period)
#         rlo, rhi = (rsi_lo, rsi_hi) if rsi_lo <= rsi_hi else (rsi_hi, rsi_lo)

#         print(f"\n{lbl}: Price[{low:,.2f} ~ {high:,.2f}] (center={center:,.2f}, half={half:,.2f})"
#               f"  |  RSI[{rlo:.2f} ~ {rhi:.2f}]")

#         # RSI 밴드별 → 가격 밴드
#         for (a,bnd) in rsi_bands:
#             band = price_band_for_rsi_band(df, a, bnd, low, high, rsi_period=rsi_period)
#             if band is None:
#                 print(f"  · RSI {a:.0f}~{bnd:.0f}: (존 RSI 범위와 겹침 없음)")
#             else:
#                 print(f"  · RSI {a:.0f}~{bnd:.0f}: Price {band[0]:,.2f} ~ {band[1]:,.2f}")

#         # 단일 RSI 타깃 → 단일 가격
#         if rsi_targets:
#             for t in rsi_targets:
#                 p = price_at_rsi_in_zone(df, t, low, high, rsi_period=rsi_period)
#                 if p is None:
#                     print(f"  · RSI={t:.1f}: (범위 밖)")
#                 else:
#                     print(f"  · RSI={t:.1f}: Price≈{p:,.2f}")

# # 예시: 각 레벨에서 RSI 40~50, 50~60 같은 밴드가 "가격으로는 어디인지" 출력
# print_fib_price_and_rsi_inversions(
#     df, res, atr_col='atr',
#     ratios=(0.786,0.618,0.5,0.382,0.236),
#     k=1.0, mode='linear', max_half_mult=1.2,
#     rsi_period=14,
#     rsi_bands=((20,30),(30,40),(40,50),(50,60),(60,70)),
#     rsi_targets=[45, 50, 55, 60]   # 원하면 특정 RSI 지점의 가격도 함께
# )



=== FIB PRICE & RSI & (RSI→PRICE) BOUNDARIES ===
Swing: up | ATR(end)=2564.330481 | k=1.0, mode=linear

R79: Price[110,325.93 ~ 114,357.06] (center=112,341.49, half=2,015.56)  |  RSI[38.99 ~ 51.26]
  · RSI 20~30: (존 RSI 범위와 겹침 없음)
  · RSI 30~40: Price 110,325.93 ~ 110,744.54
  · RSI 40~50: Price 110,744.54 ~ 113,984.23
  · RSI 50~60: Price 113,984.23 ~ 114,357.06
  · RSI 60~70: (존 RSI 범위와 겹침 없음)
  · RSI=45.0: Price≈112,544.37
  · RSI=50.0: Price≈113,984.23
  · RSI=55.0: (범위 밖)
  · RSI=60.0: (범위 밖)

R62: Price[113,720.68 ~ 116,890.19] (center=115,305.43, half=1,584.76)  |  RSI[49.07 ~ 58.39]
  · RSI 20~30: (존 RSI 범위와 겹침 없음)
  · RSI 30~40: (존 RSI 범위와 겹침 없음)
  · RSI 40~50: Price 113,720.68 ~ 113,984.23
  · RSI 50~60: Price 113,984.23 ~ 116,890.19
  · RSI 60~70: (존 RSI 범위와 겹침 없음)
  · RSI=45.0: (범위 밖)
  · RSI=50.0: Price≈113,984.23
  · RSI=55.0: Price≈115,586.36
  · RSI=60.0: (범위 밖)

R50: Price[116,105.08 ~ 118,669.42] (center=117,387.25, half=1,282.17)  |  RSI[56.41 ~ 62.26]
  · RSI 20~30

In [ ]:
# # ref['ref_price']가 min_close_in_seg와 같으면, 직전 세그먼트 내부의 최저 종가 기준이 맞게 쓰이고 있는 거야.

# # 현재 바운더리를 제외한 '직전' oversold 세그먼트 선택
# seg = pick_prev_oversold_segment_excluding_current_zone(
#     df, zones, stoch_col='close_stoch', oversold=20
# )
# print("picked seg:", seg)  # 예: (a, b)

# # 그 세그먼트 안에서 '종가 최저' ref 추출
# ref = ref_from_segment_min_close(df, seg, rsi_col='rsi')
# print(f"ref_ts={ref['ref_ts']}, ref_price={ref['ref_price']:.2f}, ref_rsi={ref['ref_rsi']:.2f}")

# # 추가 검증: 세그먼트 밖의 더 낮은 종가를 쓰지 않았는지 확인
# a, b = seg
# min_close_in_seg = float(df['close'].iloc[a:b+1].min())
# min_close_all_oversold = float(df.loc[df['close_stoch']<=20, 'close'].min())
# print("min_close_in_seg     :", min_close_in_seg)
# print("min_close_all_oversold:", min_close_all_oversold)
# # ref['ref_price'] 는 min_close_in_seg 과 같아야 함


In [ ]:
bounds, _ = build_fib_ratio_boundaries(df, res, atr_col='atr')

# 단일 타깃 값(예: need_reg)
if need_reg is not None:
    hits = level_hits_for_value(need_reg, bounds)
    print("[Regular] 레벨별 포함 여부:", {k:v for k,v in hits.items()})

# 구간(예: hid_range=(L, Xmax))
if hid_range is not None:
    L, Xmax = hid_range
    ov = level_overlaps_for_range(L, Xmax, bounds)
    print("[Hidden] 레벨별 겹침 구간:")
    for lbl, rng in ov.items():
        print(f"  {lbl}: {('없음' if rng is None else f'{rng[0]:.2f}~{rng[1]:.2f}')}")


In [ ]:
# 선행: res = fib_from_latest(df, ...), df['atr'] 존재
bounds, atr_end = build_fib_ratio_boundaries(
    df, res, atr_col='atr',
    ratios=(0.236, 0.382, 0.5, 0.618, 0.786),
    k=1.0, mode='linear',    # 또는 k=0.5, mode='sqrt'
    max_half_mult=1.2
)

# (A) 일반 불리시 타깃 값(need_reg)이 각 레벨 바운더리에 들어가는지
if need_reg is not None:
    reg_hits = level_hits_for_value(need_reg, bounds)
    print("[Regular] 레벨별 포함 여부:")
    for lbl, ok in reg_hits.items():
        print(f"  {lbl}: {ok}")
else:
    print("[Regular] 이번 봉 기준 need_reg가 None → 일반 불리시는 불가(스킵)")

# (B) 히든 불리시 구간(hid_range)이 각 레벨과 겹치는지
if hid_range is not None:
    L, Xmax = hid_range
    hid_hits = level_overlaps_for_range(L, Xmax, bounds)
    print("[Hidden] 레벨별 겹침 구간:")
    for lbl, rng in hid_hits.items():
        if rng is None:
            print(f"  {lbl}: 겹침 없음")
        else:
            lo, hi = rng
            print(f"  {lbl}: {lo:.2f} ~ {hi:.2f}")
else:
    print("[Hidden] hid_range가 없음")


In [ ]:
# 1) zones 내부에 None/누락된 low/high 있는지
bad = []
for i, z in enumerate(zones):
    if ('low' not in z) or ('high' not in z) or (z['low'] is None) or (z['high'] is None):
        bad.append((i, z))
print("깨진 존 개수:", len(bad))
if bad:
    print("예시 깨진 존:", bad[0])

# 2) need_reg / hid_range 상태
print("need_reg =", need_reg, type(need_reg))
print("hid_range =", hid_range)

# 3) 현재가/핵심 컬럼 마지막 값 상태
print("close[-1] =", df['close'].iloc[-1], type(df['close'].iloc[-1]))
print("atr[-1]   =", df['atr'].iloc[-1] if 'atr' in df else 'no atr col')


In [ ]:
# 선행: ref, hid_range, zones 혹은 (bounds) 가 이미 계산되어 있다고 가정
# ref: {'ref_price', 'ref_rsi', ...}
# hid_range: (L, Xmax)

price_now = float(df['close'].iloc[-1])
rsi_now   = float(df['rsi'].iloc[-1])
L, Xmax   = hid_range

# 1) 이번 봉에서 히든 불리쉬 가능 여부
hidden_now = (price_now > ref['ref_price']) and (rsi_now < ref['ref_rsi'])
inside     = (L <= price_now <= Xmax)
print(f"[Hidden] 이번 봉 즉시 성립? {hidden_now and inside}")
print(f" - 필요 종가 범위: {L:.2f} ~ {Xmax:.2f}")
print(f" - 현재가가 범위 내부? {inside}")

if not inside:
    target = L if price_now < L else (Xmax if price_now > Xmax else price_now)
    print(f" - 이 봉에서 성립하려면, 종가가 {target:.2f} 쪽으로 마감해야 함")

# 2) 피보나치 R23~R78 바운더리와 겹치나? (bounds가 있다면)
# bounds = build_fib_ratio_boundaries(...) 결과
hits = locate_range_overlap_in_boundaries(L, Xmax, bounds, min_overlap=0.0)
if hits:
    print("[Hidden] 겹치는 R레벨:")
    for h in hits:
        print(f"  {h['label']}: {h['overlap_low']:.2f} ~ {h['overlap_high']:.2f}")
else:
    print("[Hidden] R23~R78 바운더리와 겹침 없음")

# 3) zones 기준 컨플루언스만 보고 싶으면:
print_confluence_report(
    df, res, zones,
    need_reg=None,                # Regular는 스킵
    hid_range=hid_range,          # Hidden만 컨플루언스
    atr_col='atr',
    title="CONFLUENCE (Hidden-only)"
)
